# Talking to CSV and Excel files with LangChain

In [1]:
!pip -q install langchain openai

You should consider upgrading via the '/Users/fabian/.virtualenvs/datastuff/bin/python -m pip install --upgrade pip' command.


In [26]:
import os
import constants

# get api key
os.environ["OPENAI_API_KEY"] = constants.APIKEY

os.chdir('/Users/fabian/Desktop/langchain/')

## Dataset - Patients


In [33]:
import pandas as pd

df = pd.read_csv('data/loinc_limited.csv')

In [34]:
df.head()

,loinc_num,component,system,scale_typ,long_common_name,example_ucum_units,displayname
0,14804-9,Lactate dehydrogenase,Ser/Plas,Qn,Lactate dehydrogenase [Enzymatic activity/volu...,U/L,LDH Lactate to pyruvate reaction [Catalytic ac...
1,25542-2,Toxoplasma gondii Ab.IgM,Ser,Ord,Toxoplasma gondii IgM Ab [Presence] in Serum,NaN,T. gondii IgM Ql (S)
2,13945-1,Erythrocytes,Urine sed,Qn,Erythrocytes [#/area] in Urine sediment by Mic...,/[HPF],RBC LM.HPF (Urine sed) [#/Area]
3,2078-4,Chloride,Urine,Qn,Chloride [Moles/volume] in Urine,mmol/L,Chloride (U) [Moles/Vol]
4,711-2,Eosinophils,Bld,Qn,Eosinophils [#/volume] in Blood by Automated c...,10*3/uL,Eosinophils Auto (Bld) [#/Vol]


## CSV Agent

NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.

In [28]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [43]:
agent = create_csv_agent(OpenAI(temperature=0),
                        'data/loinc_limited.csv',
                        # ['data/patients_all.csv','data/patients.csv'],
                         model="gpt-3.5-turbo-0613",
                         verbose=True)

#so könnte prompt bearbeitet werden
agent.agent.llm_chain.prompt.template = '\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`. Each row represents on specific laboratory test which is identified by the column loinc_num. Your job is to find the best matching Loinc_num given the descriptions of a laboratory test you receive from the prompt. \nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

            

In [40]:
agent

AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df_head': '|    | loinc_num   | component                | system    | scale_typ   | long_common_name                                                                                     | example_ucum_units   | displayname                                               |\n|---:|:------------|:-------------------------|:----------|:------------|:-----------------------------------------------------------------------------------------------------|:---------------------|:----------------------------------------------------------|\n|  0 | 14804-9     | Lactate dehydrogenase    | Ser/Plas  | Qn          | Lactate dehydrogenase [Enzymatic activity/volu

In [44]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`. Each row represents on specific laboratory test which is identified by the column loinc_num. Your job is to find the best matching Loinc_num given the descriptions of a laboratory test you receive from the prompt. \nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal A


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df}

Begin!
Question: {input}
{agent_scratchpad}

In [47]:
agent.run("What is the best matching LOINC_NUM for the test: Creatinine in Plasma?")



> Entering new AgentExecutor chain...
Thought: I need to find a row in the dataframe that matches the description.
Action: python_repl_ast
Action Input: df[df['long_common_name'].str.contains('Creatinine in Plasma')]
Observation: Empty DataFrame
Columns: [loinc_num, component, system, scale_typ, long_common_name, example_ucum_units, displayname]
Index: []
Thought: I need to find a row in the dataframe that matches the description.
Action: python_repl_ast
Action Input: df[df['long_common_name'].str.contains('Creatinine in Serum')]
Observation: Empty DataFrame
Columns: [loinc_num, component, system, scale_typ, long_common_name, example_ucum_units, displayname]
Index: []
Thought: I need to find a row in the dataframe that matches the description.
Action: python_repl_ast
Action Input: df[df['long_common_name'].str.contains('Creatinine')]
Observation:      loinc_num                                       component  \
18     62238-1  Glomerular filtration rate/1.73 sq M.predicted   
24     

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5342 tokens (5086 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [32]:
agent.run("You")



> Entering new AgentExecutor chain...


KeyError: {'input', 'dfs_head', 'agent_scratchpad'}

In [11]:
agent.run("what is the distribution of age in this sample? please use Median, mean, and interquartile range")



> Entering new AgentExecutor chain...
Thought: I need to calculate the median, mean, and interquartile range of the age column
Action: python_repl_ast
Action Input: df['age'].median()
Observation: 50.565
Thought: I need to calculate the mean and interquartile range
Action: python_repl_ast
Action Input: df['age'].mean()
Observation: 48.267653631284915
Thought: I need to calculate the interquartile range
Action: python_repl_ast
Action Input: df['age'].quantile([0.25, 0.75])
Observation: 0.25    39.665
0.75    59.120
Name: age, dtype: float64
Thought: I now know the final answer
Final Answer: The median age in this sample is 50.565, the mean age is 48.268, and the interquartile range is 39.665 to 59.120.

> Finished chain.


'The median age in this sample is 50.565, the mean age is 48.268, and the interquartile range is 39.665 to 59.120.'

In [18]:
agent.run("which patientid is the lowest with an age below 45?")



> Entering new AgentExecutor chain...
Thought: I need to find the patientid with the lowest age below 45
Action: python_repl_ast
Action Input: df1[df1['age'] < 45]['patient'].min()
Observation: 80000925
Thought: I now know the final answer
Final Answer: 80000925

> Finished chain.


'80000925'

## LangChain CSV Loader

In [14]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='data/patients_all.csv')
data = loader.load()

In [ ]:
#custom chain etc